### Deriving Greeks from option spreads
Imports data from `adjusted_options` although raw data works fine as well

In [1]:
import os
import numpy as np
import pandas as pd

if os.getcwd()[-3:] == "src":
    os.chdir(os.path.dirname(os.getcwd()))
else:
    pass

**User defined parameters**

In [2]:
stock_of_interest = "AAPL"
option_data_path = "data/adjusted_options/"
default_save_path = "data/adjusted_options/"

**Load adjusted options data**

In [3]:
try:
    options_df = pd.read_csv(os.path.abspath(os.path.join(option_data_path, (stock_of_interest + ".csv"))))
    options_df["date"] = pd.to_datetime(options_df["date"]).dt.date
    options_df["expiration date"] = pd.to_datetime(options_df["expiration date"]).dt.date
except FileNotFoundError:
    raise SystemExit("Option data for " + stock_of_interest + " not found in path: " +
                     os.path.abspath(os.path.join(option_data_path, stock_of_interest)) + ".csv")

In [4]:
call_options = options_df[options_df["type"] == "call"]


Delta ($\Delta$) at a certain strike price (K) is calculated based off the forward and backward change in strike price. For example, for K=20, the delta at K=19.5 is calculated by taking the slope between K=19 and K=20. Likewise, Delta at K=20.5 is based off ask/bid prices at K=20 and K=21.

Note that the assumption of midpoint is built upon the existence of a monotonic Delta function, which is realistic. For example, the Delta of long calls should decrease from 1 (very ITM) to 0 (very OTM).

In [ ]:
dates = set(call_options["date"])

for my_day in dates:
    day_options = call_options[call_options["date"] == my_day]
    exp_dates = set(day_options["expiration date"])
    for exp_day in exp_dates:
        exp_day_options = day_options[day_options["expiration date"] == exp_day]
        ask_delta_df = exp_day_options[["strike price", "ask price"]].reset_index(drop=True) - \
                       exp_day_options[["strike price", "ask price"]][1:].reset_index(drop=True)
        ask_delta_df = ask_delta_df.rename(columns={"strike price": "d_strike","ask price": "d_ask"})
        ask_delta_df["avg strike"] = (exp_day_options["strike price"].reset_index(drop=True) + \
                                     exp_day_options["strike price"][1:].reset_index(drop=True)) / 2

        ask_delta_df["delta"] = ask_delta_df["d_ask"] / (-ask_delta_df["d_strike"])

        print("stop")